# HuggingFace와 익숙해지기 🤗

**Machine Reading Comprehension에 입과하신 캠퍼분들 반갑습니다 ! 👋🏻💖**

'KLUE' 강의를 통해 Huggingface에 이미 익숙해지셨을거라 생각하지만

빠르게 Huggingface를 리마인드 하겠습니다 :)

본 Mission은 MRC 강의를 진행하기 전 Huggingface에 대해 짧게 복습하는 것이니, 부담없이 코드를 돌려보시면 됩니다 😊


* 아래의 링크에서 더 자세하고 꼼꼼한 튜토리얼을 공부할 수 있습니다. 참고해보세요 🤗

    https://huggingface.co/transformers/notebooks.html


* 아래의 링크에서 huggingface에서 제공하는 라이브러리에 대한 discussion을 살펴볼 수 있습니다. 

  https://discuss.huggingface.co/


```
🛠 Setup을 하는 부분입니다. 무지성 실행 하셔도 좋습니다.
📣 설명이 적혀있는 부분입니다. 빠르게 읽어보면서 이해하시면 됩니다.
💻 실습 코드입니다. 따라가면서 코드를 이해해보세요.
```

##🛠 초기 설정

### 🛠  Requirements

mrc 베이스라인과 버전 통일을 위해 transformers 4.24.0 을 사용합니다

In [ ]:
! pip install transformers==4.24.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00


## 💻 Huggingface transformers 빠르게 훑어보기

### 📣 소개

<center><img src='https://raw.githubusercontent.com/yukyunglee/yukyunglee.github.io/master/HF.png' width=500>


NLP 모델은 아래와 같은 Pipeline으로 구성됩니다

`Input` -> `Tokenization` -> `Model training/Inference` -> `Post-Processing` (task dependent) -> `Output`

이러한 pipeline을 모두 scratch부터 구현해보는것은 매우 의미있겠지만, 

task에 알맞는 모델을 빠르게 구현하기 위해서는 반복적인 작업을 **정형화**할 필요성이 있습니다.

🤗 **Huggingface transformers** 🤗는 모델을 쉽고 빠르게 생성하고, 학습하고, 배포하기 위해 만들어진 Highlevel Library 입니다.

즉, 우리는 Huggingface를 사용함으로서 반복되던 작업을 간편하게 수행할 수 있고, **모델 개발에 초점을 맞추어 개발 및 연구**를 진행할 수 있게됩니다.



### 💻 Tokenizer 불러오기

텍스트 데이터를 모델이 알아들을 수 있는 형태로 변환하기 위해서는 `tokenization` 과정을 거쳐 encoding을 진행해야합니다. 

이 때 text data를 token화 하고 특정 숫자로 encoding 하는 과정을 모두 수행하는것이 transformers tokenizer 역할입니다.

> Q : PLM(Pretrained language model)dl 없었던 시기에는 tokenizer를 어떻게 사용했을까요 ? 

> A : 내가 가지고 있는 데이터를 기반으로 parsing을 진행(성능 좋은 Parser를 사용했음)한 후 dictionary를 데이터별로 만들어 직접 숫자를 부여했습니다



In [ ]:
from transformers import AutoTokenizer

example = "조대현은 최고의 조교이다."

# tokenization 결과를 보여줍니다.
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# tokenization 결과를 보여줍니다.
# bert-base-cased는 영어에 대한 tokenizer이므로 한글을 전혀 이해하지 못함
# 따라서 tokenizer output은 [unk]으로 나옴
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['[UNK]', '[UNK]', '[UNK]', '.']
tokenization + encoding 결과 :  [101, 100, 100, 100, 119, 102]


In [ ]:
example = "이승윤는 최고의 조교이다."

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# tokenization 결과를 보여줍니다.
# klue/bert-base는 한글에 대한 tokenizer이므로 한글을 인식할 수 있음
# 언어에 알맞는 적절한 tokenizer를 사용해야함
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['박성', '##호', '##는', '최고', '##의', '멘토', '##이다', '.']
tokenization + encoding 결과 :  [2, 10649, 2016, 2259, 3841, 2079, 8843, 28674, 18, 3]


**⛔️ tokenizer 사용시 주의사항**

1. train data의 언어를 이해 할 수 있는 tokenizer인지 확인
2. 사용하고자 하는 pretrained model과 동일한 tokenizer인지 확인
  
  > 적절한 tokenizer를 사용하지 않을 경우 **vocab size mismatch**에러가 발생하거나 **special token이 `[unk]`**으로 처리되는 🤦🏻‍♀️대참사🤦🏻‍♂️가 벌어질 수 있음
3. 단어의 개수와 special token이 완전히 일치하는 모델은 (예를들어 klue의 roberta, bert) tokenizer를 cross로 사용 **'할 수도'** 있지만 옳은 방법은 아님
  * 첨언하자면, 공개된 영어 bert와 roberta는 tokenizer가 호환되지 않습니다. (bert vocab 28996개, roberta vocab 50265개)
  * klue bert는 동일한 기관에서 생성된 모델이므로 32000개로 총 vocab 사이즈가 동일하지만 이는 우연의 일치입니다.



------

### 💻 Config 불러오기

**사전 학습 모델을 사용하기 위해서는 사전학습 모델이 가진 setting을 그대로 가져와야합니다.**

모델마다 vocab size, hidden dimension등 각각의 파라미터 세팅이 상이하므로 

transformers는 이 정보를 Config로 쉽게 불러올 수 있는 기능을 제공합니다.

모델명+Config.from_pretrained가 가장 기본적인 형태였지만, 요즘은 `Auto` class로 더욱 편리하게 configuration을 가져올 수 있습니다.




In [ ]:
from transformers import AutoConfig

model_name =  'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

In [ ]:
model_config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

**⛔️ config 사용시 주의사항**

어떤 경우에는 config를 수정하여 사용하기도 하는데, 바꾸어도 되는 config와 바꾸지 말아야 하는 config가 정해져 있습니다.

**바꾸면 안되는 config**
* Pretrained model 사용시 hidden dim등 이미 정해져 있는 모델의 아키텍쳐 세팅은 수정하면 안됩니다.
* 이를 수정해버릴 경우 에러가 발생하거나, 잘못된 방향으로 학습 될 수 있습니다.

**바꾸어도 되는 config**
* vocab의 경우 special token을 추가한다면 config를 추가한 vocab의 개수만큼 추가하여 학습해야합니다.
* downstream task를 위해 몇가지 config를 추가할 수도 있습니다. (아래에서 예시를 살펴봅시다)






**보다 빠른 이해를 돕기 위해, KLUE 강의에서 사용하셨던 sequence classification 모델을 config 세팅의 예시로 설명합니다.**

transformers document를 보면 사용하는 모델별로 미리 정의되어야 하는 config들에 대해 알려주고 있습니다

```python
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
```
우리의 경우엔 num_labels라는 정보를 꼭 기입해주어야 하는 상황입니다. 

그럼 아래의 예시를 살펴볼까요 ?


In [ ]:
'''
현재 상황은 아래와 같습니다.
1) 사용 모델 : Sequence classification 모델을
2) special token 2개 추가함
3) label은 총 10개

case 1. 원하는 config의 값을 수정하는 케이스 (2)
case 2. downstream task를 위해 추가해야하는 config 케이스 (3)
'''

model_name = 'klue/bert-base'

# case 1
# config를 추가하는 방법은 두가지로 가능함 

# [1] 호출 후 직접 config의 값을 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name)
model_config.vocab_size = model_config.vocab_size + 2
print('case 1 - [1] : ', model_config)

# [2] 호출과 동시에 수정하는 방법 
# 하지만, vocab 수정은 해당 코드로 진행하는것을 권장하지 않음
# Advanced tutorial의 token 추가하기에서 자세히 다룸
model_config = AutoConfig.from_pretrained(model_name , vocab_size=32002)

print('case 1 - [2] : ', model_config)

case 1 - [1] :  BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32002
}

case 1 - [2] :  BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 5

In [ ]:
# case 2 (sequence classification을 위해 num_labels를 설정하기)
# config를 추가하는 방법은 두가지로 가능함 

# [1] 호출 후 직접 config의 값을 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name)
model_config.num_labels = 10

# print('case 1 - [1] : ', model_config)

# [2] 호출과 동시에 수정하는 방법
model_config = AutoConfig.from_pretrained(model_name , num_labels=10)

# print('case 1 - [2] : ', model_config)

📣 **아무런 값이나 config에 추가할 수 있을까요 ?**

임의로 만들어진 config key는 config에 추가 될수는 있지만 모델 학습에 사용되지 않습니다.

In [ ]:
# 해당 방법으로 생성하면 config에 추가 됨
model_config.hey = 'hey~'

model_config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hey": "hey~",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,

In [ ]:
# 해당 방법으로 생성하면 config에 추가되지 않음 (모델 학습에 사용되지 않는 key 이므로)
# AutoConfig.from_pretrained(model_name/path , **kwargs)
# model name 이후에 keyward argument(**kwargs)로 인자를 받을 수 있으니 딕셔너리 형태로 값을 받을 수도 있습니다. 
model_config = AutoConfig.from_pretrained(model_name , hey="hey~")
model_config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
# 참고 : 원하는 custom_config로 config 전체를 업데이트 해주는 방법도 있음

custom_config = {
    # 원하는 값을 넣으면 됩니다
    "vocab_size": 32000
}

model_config = AutoConfig.from_pretrained(model_name)
model_config.update(custom_config)

---

### 💻 Pretrain model 불러오기

**transformers의 가장 강력한 기능은 사전학습된 모델을 쉽게 불러오고, 사용할 수 있다는 것입니다.**

단 세 줄로 원하는 pretrained model을 불러오고 사용할 수 있습니다.

(물론 원하는 모델이 Huggingface transformers에 공개되어 있어야하겠죠 !😎)

우리는 해당모델을 그대로 사용할 수도 있고, 추가적으로 학습을 진행하여 내 데이터에 맞는 모델로 사용할수도 있습니다.

* `.from_config()` 는 config 그대로 모델을 가져오는 method 입니다. 즉 사전학습된 weight을 가져오는게 아니니 주의해야합니다.
* `.from_pretrained()` 는 model config에 해당하는 모델을 가져오고, **사전학습된 weight를 가져옵니다**. 스스로 학습한 모델을 불러오려면 model_name 부분에 model이 저장된 directory를 입력하면 됩니다.



**transformers는 두가지 타입의 모델을 제공하고 있습니다.**

* 기본 모델
  
  : hidden state가 출력되는 기본 모델

* downstream task 모델
  
  : 일반적인 task를 쉽게 수행할 수 있도록 미리 기본 모델 + head가 설정된 모델
  
  : output은 task에 적합한 dimension으로 미리 정의되어있음

**+ ) transformers 사용자들이 공개한 checkpoint를 마음껏 사용할 수 있습니다.**

  https://huggingface.co/models

  models에서 이미 배포된 모델을 손쉽게 사용할 수 있으니 목적에 맞는 모델을 찾아 사용해보는것도 좋습니다.

In [ ]:
from transformers import AutoConfig, AutoModelForQuestionAnswering
# Download configuration from huggingface.co and cache.

model_name = 'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

# 모델을 정의합니다.

# option 1 : config에서 정의한 모델을 가져오기 (initial)
# model = AutoModelForQuestionAnswering.from_config(config)

# option 2 : config에서 정의한 사전학습된 모델을 가져오기 (pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(
        model_name, config=model_config
    )

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

**이제 우리는 가장 기본적인 tokenizer, config, model를 loading하는 방법에 대해 배웠습니다.**

원리를 알게 되었으니 아래의 방법으로 세가지를 한번에 load하면 됩니다.

In [ ]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

----

## 📣 Huggingface Trainer 
> Trainer is all you need ? 🤔

  반복되는 Training loop를 효과적으로 모듈화 시켜놓은것이 transformers의 trainer입니다. 
  
  덕분에 우리는 매 모델을 학습하기 위해 training loop를 구현하는 과정을 단 몇줄만에 해결할 수 있습니다.


아래와 같은 structure로 trainer를 사용할 수 있습니다.

* TrainingArguments 설정
* Trainer 호출
* 학습 / 추론 진행

```python
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = 'klue/bert-base'

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
```



RE 대회를 잘 수행하셨다면 기본적으로 Trainer를 사용하는 방법은 모두 아실거라고 생각합니다.

 **하지만 Trainer가 '항상' 좋을까요 ?**

* **다음과 같은 모듈의 코드는 legacy가 존재할 수 밖에 없습니다.** 

  : 따라서 버전이 바뀔 때 마다 변동되는 사항이 많아지고 코드를 지속적으로 수정해야하는 단점이 존재합니다. 

  : pytorch lightning이 대표적으로 이러한 문제를 겪고 있으며, transformers도 예외는 아닙니다.

  : 따라서 Trainer는 모든 상황에서 정답이 될 수는 없습니다

* **최대한 편리함을 이용하되, 동작 원리를 살펴보는 과정이 매우 중요합니다.**

* Trainer의 구조를 살펴보고, 내가 학습할 모델을 위한 Trainer를 만들어보는것도 좋은 방법입니다 
  * Trainer에 원하는 함수 오버라이딩 하여 수정하기 (general task에 적합)
  * Custome Trainer 만들어보기 (general task가 아닌경우 유용함)

## 💻 Advanced tutorial

### 💻 Token 추가하기

간혹 모델의 성능을 높이기 위해 special token을 추가하거나, domain에 특화된 단어를 추가해주는 방법이 있습니다. 

* special token을 추가하는 경우 해당 token이 special token임을 tokenizer에게 알려주어야 합니다.

  : 따라서 이 경우에는 `add_special_tokens()` 메서드를 사용해야합니다.

* 일반 token을 추가하는 경우엔 `add_tokens()` 메서드를 사용하여 vocab을 늘려줄 수 있습니다.

* tokenizer에 vocab을 추가했다면 pretrained model의 token embedding 사이즈를 변경해주어야합니다.

  : `model.resize_token_embedding`을 이용하면 됩니다.
  
  : tokenizer는 `len()` 사용하면 vocab의 총 개수가 나오므로 이를 이용하면 됩니다

  : 추가한 개수 만큼 vocab을 늘려주고, embedding 사이즈도 늘려주는 과정을 통해 직관적으로 vocab을 추가합니다
 



* Reference code

  [special token 추가하기](https://github.com/huggingface/tokenizers/issues/247#issuecomment-675458087)

  [token 추가하기](https://medium.com/@pierre_guillou/nlp-how-to-add-a-domain-specific-vocabulary-new-tokens-to-a-subword-tokenizer-already-trained-33ab15613a41)


In [ ]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# special token 추가하기 
special_tokens_dict = {'additional_special_tokens': ['[special1]','[special2]','[special3]','[special4]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# token 추가하기
new_tokens = ['COVID', 'hospitalization']
num_added_toks = tokenizer.add_tokens(new_tokens)

# 기존 config로 모델을 불러오기
# 모델을 불러오기전에 vocab을 수정하면 pretrained config와 충돌이 일어나 에러가 발생하니 주의
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

# tokenizer config 수정해주기 (추후에 발생할 에러를 줄이기 위해)
config.vocab_size = len(tokenizer)

# model의 token embedding 사이즈 수정하기
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

Embedding(32006, 768)

**Q : special token을 추가할 때 항상 resize를 해주어야 하나요 ?**

A : 꼭 그렇지 않습니다. 잘 만들어진 모델은 resize를 하지않고도 모델에 새로운 vocab을 추가할 수 있도록 여분의 vocab 자리를 만들어 두었습니다. *여분의 vocab 개수는 모델에 따라 다르니 확인이 필요합니다.*

아래의 코드를 살펴봅시다

In [ ]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.vocab["[unused0]"]

31500

In [ ]:
# klue/bert-base는 500개(index:0~499) dummy vocab을 가지고 있음
tokenizer.vocab["[unused499]"]

31999

`tokenizer.vocab`에 특정 단어가 포함되어있을지 확인해보려면 단어를 string 타입으로 넣어서 확인해보면 됩니다. 

`[unused0]`이라는 인풋이 보시이나요 ? 사용하지 않는 dummy vocab을 추가했다는 의미입니다. 

* **이러한 dummy vocab을 추가하는 이유는 무엇일까요 ?**
  : 사용자의 니즈에 따라서 단어를 추가할 수 있는 여유 공간을 제공한 것입니다. 즉, 유저가 pretrained model을 최대한 수정하지 않고도 다양한 vocab을 사용할 수 있도록 의도한 것입니다.

* 최근에 공개된 모델들은 대부분 'unused' vocab을 가지고 있습니다.

* 즉, **모든 모델이 dummy vocab을 고려하는것은 아닙니다.**
  * 예를들어, SKT의 KoBERT는 dummy vocab을 가지고 있지 않습니다.
  * 따라서 추가 vocab을 넣을 경우에는 manual 하게 수정을 해줘야 하며, gluonnlp를 사용하는 부분을 수정해야합니다. 
  * 어떻게 알았냐구요 ? 저도 알고 싶지 않았습니다. 🙄


In [ ]:
model_name = "bert-base-cased"

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# bert-base-cased 모델은 100개(index :1~101)의 dummy vocab을 가지고 있음
tokenizer.vocab["[unused101]"]

105

모델별로 dummy vocab을 위한 자리가 미리 마련된걸 알았으니, tokenizer loading시 cache가 저장되는 디렉토리로 이동해서 vocab.txt 파일을 manual하게 변경해주면 resize 없이 사용할 수 있습니다. (귀찮다면 add_token 후 resize를 합시다)

* vocab을 매우 많이 추가했다면 pretraining을 다시 수행하는것이 좋습니다 (TAPT: Task Adaptive PreTraining)

  * TAPT는 아래의 논문을 참고하세요 🙂

    : [Don't Stop Pretraining: Adapt Language Models to Domains and Tasks](https://arxiv.org/abs/2004.10964)

* 그렇지 않다면 finetuning만 수행해도 충분합니다

### 💻 [CLS] output 추출하기

model에서 [CLS] 자리의 embedding만 가지고 오고 싶은 경우가 있습니다. 

이때 전체 output representation에서 indexing으로 `[CLS]`embedding을 가지고 올 수 도 있지만 

`.pooler_output` 을 이용하면 보다 쉽게 값을 가져올 수 있습니다


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

inputs = tokenizer("서민준 교수님이 MRC를 뒤집어 놓으셨다 !!", return_tensors="pt")
outputs = model(**inputs)

cls_output = outputs.pooler_output

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### 📣 참고 : [CLS] 토큰은 정말 문장을 대표할까 ?

> 해당 파트는 참고용입니다 :) 관심이 있으시다면 읽어보셔도 좋습니다.

BERT paper를 살펴보면 [CLS] 토큰은 문장을 대표하는 값으로 알려져 있습니다. 

**의심의 여지 없이 문장을 대표할까요 ?**

놀랍게도, BERT의 저자 또한 [CLS]가 Sentence representation이란걸 보장할 순 없다고 밝혔습니다.

아래의 이슈를 보시면 매우 흥미로운 주제로 이야기를 나누고 있습니다
https://github.com/google-research/bert/issues/164

* 우리가 특정 task를 수행할 때 `[CLS]` 토큰이 '당연히' 문장을 대표해줄것이라는 가정을 가지는것은 위험합니다.

* 실험을 통해 여러분이 수행하고자 하는 task에 어떤 값이 중요할지 확인해보세요 ! 

* **읽어볼만한 논문 추천 (SBERT)**
  
  : 논문 : https://arxiv.org/pdf/1908.10084.pdf

  : ***The most commonly used approach is to average the BERT output layer (known as BERT embeddings) or by using the output of the first token (the [CLS] token). As we will show, this common practice yields rather bad sentence embeddings, often worse than averaging GloVe embeddings (Pennington et al., 2014).***

  : 요약) avg나 CLS를 사용하는게 일반적이지만 이건 Glove
  embedding의 avg보다도 성능이 낮다.

  : 저는 해당 논문을 읽고 `[CLS]`가 sentence 를 대표하지 못하겠구나를 처음 인지하게 되었습니다. 

  : input에 대한 representation을 추출한 후 pooling layer를 쌓아 maxpooling이나 average pooling을 수행하기도 합니다.



### [TIP!] 🚪✊Knock Knock을 활용하여 모델 학습 완료 알림받기

🚪✊Knock Knock

* 학습 후 메일, 슬랙 등 원하는 곳으로 학습 종료 알람을 해주는 라이브러리입니다

* huggingface에서 만든 공식 라이브러리니 사용해보시는걸 추천드립니다

* 더 자세한 내용은 아래 링크를 참고하세요 !
https://github.com/huggingface/knockknock

* 아래의 코드를 삽입하면 학습 후 슬랙 알림을 받을 수 있습니다

> pip install knockknock

```python
from knockknock import slack_sender

webhook_url = "<webhook_url_to_your_slack_room>"
@slack_sender(webhook_url=webhook_url, channel="<your_favorite_slack_channel>")
def train_your_nicest_model(your_nicest_parameters):
    import time
    time.sleep(10000)
    return {'loss': 0.9} # Optional return value

```


command line에서 실행하기
```bash
knockknock slack \
    --webhook-url <webhook_url_to_your_slack_room> \
    --channel <your_favorite_slack_channel> \
    sleep 10

```



### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

